In [1]:
#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
from numpy import int64
from numpy import float64
from numpy import datetime64
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
# Allows plots to appear directly in the notebook.
%matplotlib inline
# Date/time functionality
import datetime
import time
# Check if files exist
from os.path import exists
from os import makedirs
# System specific parameters and functions
import sys

import os
import glob

import warnings
warnings.filterwarnings("ignore")
import multiprocessing 
from multiprocessing import Pool ,cpu_count

In [ ]:
dtype={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','VEHICLEID':'category','SUPPRESSED':'category','JUSTIFICATIONID':'category','LASTUPDATE':'str','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','NOTE':'str','DISTANCE':'int64','MINUTES':'int64'}
mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")

In [ ]:
    """location, dtypes, datecolumns ,func"""
arg_list=['/Chunks/DBUS/LINEID_CHUNKS_CVS/',dtype,['DAYOFSERVICE','LASTUPDATE'],mydateparser]

In [ ]:
LINEID_file = "LINEID.csv"
df_LINEID = pd.read_csv(LINEID_file, low_memory=False)
LINEID_array=df_LINEID['LINEID'].unique()
LINEID_list=LINEID_array.tolist()

In [2]:
def get_chunks(x):
    """x=[location, dtypes, datecolumns ,func]"""
    cwd=os.getcwd()
    path = cwd+x[0]
    chunk_folder = glob.glob(os.path.join(path, "*.csv"))
    chunk_list=[]
    counter=0
    for filename in chunk_folder:
        # read the csv file
        df_chunk = pd.read_csv(filename, low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3])
        chunk_list.append(df_chunk)
        print('Index at chunk_list',counter,' is ', filename, ': ',df_chunk.shape)
        counter+=1
    return chunk_list

Reading csv files at location to chunk of dataframes by casting to appropriate datatypes and addinf dataframes to a list. 

In [5]:
chunk_dbus_list=get_chunks(arg_list)

Index at chunk_list 30  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/66B.csv :  (484826, 20)
Index at chunk_list 31  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/43.csv :  (803190, 20)
Index at chunk_list 32  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/70.csv :  (666621, 20)
Index at chunk_list 33  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/51X.csv :  (11179, 20)
Index at chunk_list 34  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/25X.csv :  (49084, 20)
Index at chunk_list 35  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/54A.csv :  (806756, 20)
Index at chunk_list 36  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/27X.csv :  (41458, 20)
Index at chunk_list 37  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/33X.csv :  (141114, 20)
Index at chunk_list 38  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/53.csv :  (170461, 20)
Index at chunk_list 39  i

Index at chunk_list 106  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/31B.csv :  (125557, 20)
Index at chunk_list 107  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/17.csv :  (1171068, 20)
Index at chunk_list 108  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/111.csv :  (348247, 20)
Index at chunk_list 109  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/41D.csv :  (1117, 20)
Index at chunk_list 110  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/42D.csv :  (11079, 20)
Index at chunk_list 111  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/84.csv :  (998621, 20)
Index at chunk_list 112  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/15B.csv :  (1639422, 20)
Index at chunk_list 113  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/38D.csv :  (13461, 20)
Index at chunk_list 114  is  /home/team8/notebook/Pelin/Chunks/DBUS/LINEID_CHUNKS_CVS/33D.csv :  (8760, 20)
Index at chunk_l

In [6]:
len(chunk_dbus_list)

130

In [ ]:
chunk_dbus_list[0].head(5)

In [7]:
chunk_dbus_list[0].dtypes

DAYOFSERVICE          datetime64[ns]
TRIPID                      category
PROGRNUMBER                    int16
STOPPOINTID                 category
PLANNEDTIME_ARR                int64
PLANNEDTIME_DEP                int64
ACTUALTIME_ARR                 int64
ACTUALTIME_DEP                 int64
VEHICLEID                   category
SUPPRESSED                  category
JUSTIFICATIONID             category
LASTUPDATE            datetime64[ns]
DIFFERENCETIME_ARR             int64
DIFFERENCETIME_DEP             int64
LINEID                      category
ROUTEID                     category
DIRECTION                   category
NOTE                          object
DISTANCE                       int64
MINUTES                        int64
dtype: object

Creating a dataframe by mapping lineid with list index 

In [ ]:
def line_id_list_index_matcher(dbus_list):
    df_c_dbus_list_index_lineid = pd.DataFrame()
    index_count=0
    for df in chunk_dbus_list:
        a=df['LINEID'].unique()
        df_c_dbus_list_index_lineid=df_c_dbus_list_index_lineid.append({'List_index':index_count,'LINEID':a[0]},ignore_index=True)
        index_count+=1
    return df_c_dbus_list_index_lineid

### Data Quality Plan

|Feature|	Data Quality Issue|	Handling Strategy|
| -: | -: | -: |
|DAYOFSERVICE| None|This is main constructor of time series will be kept to derive additional features which might have effect on predicting travel times, such as months. Also might be used to convert planned/actual arrival/departure times into date/time objects to extract data about which hour of day and which day of week, the bus was at the bus stop in data at the row. After extracting those new features this column can be dropped.
|TRIPID|None | There is information about 658961 Trips in this dataset. This means there is approximately 116949113/658961=177 row of data given per trip on average. Need to check if we can use this feature to derive information about secondary delays. Trips distance etc.  If we combine TRIPID with ProgramNumber then we will know which row points to one previous stop and delay in that stop is basically the measure of secondary delay. After combining with program number to extract that new feature this column can be dropped.
|PROGRNUMBER|None| sequential position of stop point in the trip. Combined with TripID can be used to extract secondary delays.Exp: in order to find secondary delay at tripno 30 progrnumber 2, we basically need to check DIFFERENCETIME_DEP at  tripno 30 progrnumber 1. After combining with program number to extract that new feature this column can be dropped.
|STOPPOINTID| None|There is 4774 stop points in this dataset. It is same stop point at google maps and TFI app.        
|PLANNEDTIME_ARR|None | This can be used to figure out hour of day this stop event occur but the problem is we don't know what time trip started.We can assume all trips started at 00:00. Checked to see if it made sense on this data there is a record of bus no 15b being at bus stop no 1018 on a sunday at 46917 sec this corresponds to 13:01:57 if trip stated at 00:00, when I compare this data with **current** time table of 15b at bus no 1018(http://www.dublinbus.ie/Your-Journey1/Timetables/All-Timetables/15b-/) I can see a planned arrival at 13:05. I think minutes are rounded to multipliers of 5 while printing out time tables so 13:01 might be rounded 13:05 on this published time table. In order to find out how long it take for bus to reach this proportion of trip in seconds we will use formula (time of trip at bus stop-begining time of trip)/(ending time of trip-beginning time of trip)
|PLANNEDTIME_DEP|None |   "           
|ACTUALTIME_ARR|None |   "          
|ACTUALTIME_DEP|None |   "          
|VEHICLEID|Unique identifier not to be used as feature in data preparation | There is 1151 vehicle in this dataset. This is categorical feature and don't think it will effect travel times as same vehicle can be assigned to different route or can be used by a different driver at different trips so logically this have little to no effect on travel time prediction, dropping.
|SUPPRESSED| categorize an outlier| Suppressed flag data is only 0.5% of this dataset. and since supressed trips are cancled ones we don't want to have them in our data they are outliers so those **rows** will be dropped. 
|JUSTIFICATIONID|Unique identifier not to be used as feature in data preparation| This is a key which points out to another table in dataset which we don't have access to so irrelevant to our target prediction therefore this feature will be dropped.   
|LASTUPDATE| Unique identifier not to be used as feature in data preparation | this data is not related to bus travel times it is an input for source databases record tracebility therefore this feature will be dropped.
|DIFFERENCETIME_ARR|  None | Keep as is  
|DIFFERENCETIME_DEP|  None | Keep as is  
|LINEID|  None | Keep as is , It is same lineID at google maps and TFI app.  check if LINEID+DIRECTION+STOPPOINTID corresponds to one or multiple PROGRNUMBER  
|ROUTEID|None | check if each ROUTEID+STOPPOINTID corresponds to one or multiple PROGRNUMBER       
|DIRECTION|None | Keep as is ,         
|NOTE| Too many missing values  | will be dropped.   
|DISTANCE| irrelevant to prediction target | will be dropped.                  
|MINUTES| irrelevant to prediction target |  will be dropped.              

##### Changes in implementation of above plan: 
* We decide to derive secondary delays from TRIPID and  PROGRNUMBER and use it in our model as feature. But thinking about it if we use this feature in our model then in order to run the model,  if user enter a date/time for the trip that haven't start yet we need to assume secondary delay is 0, this assumption will be reducing model accuracy and for the trips that already start we will need to use Dublin Bus Live Updates from NTA API to obtain current arrival time and need to find a way to compare it with data at time table (which will mean finding current time table data from somewhere and enter in to database) to calculate secondary delay which is too much cost for benefit. So we are passing extract secondary delay feature step at quality plan and this is the justification for the decision. 

#### 1-)Dropping columns won't be transformed or used to derive features
    1- VEHICLEID , JUSTIFICATIONID , LASTUPDATE , NOTE, DISTANCE, MINUTES
    2- Important note for further steps: * Keeping LINEID for now, even if it isn't an model input it is an important parameter will be dropped after further cleaning the datasets.* 

In [8]:
for df in chunk_dbus_list:
    df.drop(['VEHICLEID','JUSTIFICATIONID','LASTUPDATE','NOTE','DISTANCE','MINUTES'],axis=1,inplace=True)

In [9]:
chunk_dbus_list[129].head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,DIFFERENCETIME_ARR,DIFFERENCETIME_DEP,LINEID,ROUTEID,DIRECTION
0,2018-12-06,8161457,2,4348,27702,27702,27626,27676,NaN,-76,-26,49,49_28,2
1,2018-12-06,8161457,11,4756,28470,28470,28568,28584,NaN,98,114,49,49_28,2
2,2018-12-06,8161457,13,4757,28621,28621,28748,28780,NaN,127,159,49,49_28,2
3,2018-12-06,8161457,16,4759,28751,28751,28962,28962,NaN,211,211,49,49_28,2
4,2018-12-06,8161457,19,2548,28966,28966,29166,29188,NaN,200,222,49,49_28,2


In [10]:
chunk_dbus_list[129].shape

(988486, 14)

In [11]:
def write_to_csv(adress,dbus_list ):
    cwd=os.getcwd()
    for df in dbus_list:
        a=df['LINEID'].unique()
        file_name=a[0]
        path = cwd+adress
        df.to_csv(path+file_name+'.csv',index=False)
        print('Location:', path+file_name+'.csv')

In [12]:
write_to_csv('/Chunks/DBUS/QP_Implementation/',chunk_dbus_list)

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/239.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/151.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/70D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/1.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/123.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/61.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/161.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/33E.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/66.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/67X.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/41A.csv
Location: /home/team8/notebook

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/84.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15B.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/38D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/33D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/79A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/67.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/17A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/16D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/38B.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/46A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/29A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/185.csv
Location: /home/team8/noteb

#### 2-) Extract information about which hour of day and which day of week entery created. 
    * Add number of seconds information given at ACTUALTIME_ARR time to DAYOFSERVICE assume trip began at 00:00 

In [ ]:
dtype={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','SUPPRESSED':'category','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category'}
mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
arg_list=['/Chunks/DBUS/QP_Implementation/',dtype,['DAYOFSERVICE'],mydateparser]

In [ ]:
chunk_dbus_list=get_chunks(arg_list)

In [13]:
for df in chunk_dbus_list:
    df['ACTUALTIME_ARR_DATETIME'] = pd.to_datetime(df['DAYOFSERVICE']) + pd.to_timedelta(df['ACTUALTIME_ARR'], unit='s')

In [14]:
chunk_dbus_list[129].head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,DIFFERENCETIME_ARR,DIFFERENCETIME_DEP,LINEID,ROUTEID,DIRECTION,ACTUALTIME_ARR_DATETIME
0,2018-12-06,8161457,2,4348,27702,27702,27626,27676,NaN,-76,-26,49,49_28,2,2018-12-06 07:40:26
1,2018-12-06,8161457,11,4756,28470,28470,28568,28584,NaN,98,114,49,49_28,2,2018-12-06 07:56:08
2,2018-12-06,8161457,13,4757,28621,28621,28748,28780,NaN,127,159,49,49_28,2,2018-12-06 07:59:08
3,2018-12-06,8161457,16,4759,28751,28751,28962,28962,NaN,211,211,49,49_28,2,2018-12-06 08:02:42
4,2018-12-06,8161457,19,2548,28966,28966,29166,29188,NaN,200,222,49,49_28,2,2018-12-06 08:06:06


In [15]:
chunk_dbus_list[129].shape

(988486, 15)

In [16]:
write_to_csv('/Chunks/DBUS/QP_Implementation_backup/',chunk_dbus_list)

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/239.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/151.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/70D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/1.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/15D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/123.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/61.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/161.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/15.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/33E.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/66.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/67X.csv
Location: /home/team8

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/51D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/25D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/27A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/33.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/40.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/31B.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/17.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/111.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/41D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/42D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/84.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/15B.csv
Location: /home/team

#### 3-) Extract information about how long will it take bus to reach this proportion of trip and what is the total journey time for this trip
* proportion of trip use formula; (time of trip at bus stop-begining time of trip)/(ending time of trip-beginning time of trip)
    * begining time of trip: to extract this we need to find min value of ACTUALTIME_ARR where TRIPID is equal to TRIPID of row
    * ending time of trip: to extract this we need to find max value of ACTUALTIME_ARR where TRIPID is equal to TRIPID of row
* total journey time: ending time of trip-beginning time of trip

In [17]:
chunk_dbus_list_v1=[]
index_tracker=0
for df in chunk_dbus_list:
    df_begining_time_of_trip=pd.DataFrame(df.groupby(['TRIPID'])['ACTUALTIME_ARR'].min())
    df_begining_time_of_trip.rename(columns={'ACTUALTIME_ARR':'ACTUALTIME_ARR_MIN'},inplace=True)
    print("for df at index ",index_tracker," begining_time_of_trip ",df_begining_time_of_trip.shape)
    df=pd.merge( df, df_begining_time_of_trip, on="TRIPID", how="left")
    print("for df at index ",index_tracker," after merging with beginning t ",df.shape)
    df_ending_time_of_trip=pd.DataFrame(df.groupby(['TRIPID'])['ACTUALTIME_ARR'].max())
    df_ending_time_of_trip.rename(columns={'ACTUALTIME_ARR':'ACTUALTIME_ARR_MAX'},inplace=True)
    print("for df at index ",index_tracker," ending_time_of_trip ",df_ending_time_of_trip.shape)
    df=pd.merge( df, df_ending_time_of_trip, on="TRIPID", how="left")
    print("for df at index ",index_tracker," after merging with ending t ",df.shape)
    chunk_dbus_list_v1.append(df)
    index_tracker+=1

for df at index  0  begining_time_of_trip  (1875, 1)
for df at index  0  after merging with beginning t  (205531, 16)
for df at index  0  ending_time_of_trip  (1875, 1)
for df at index  0  after merging with ending t  (205531, 17)
for df at index  1  begining_time_of_trip  (10318, 1)
for df at index  1  after merging with beginning t  (1575364, 16)
for df at index  1  ending_time_of_trip  (10318, 1)
for df at index  1  after merging with ending t  (1575364, 17)
for df at index  2  begining_time_of_trip  (66, 1)
for df at index  2  after merging with beginning t  (13584, 16)
for df at index  2  ending_time_of_trip  (66, 1)
for df at index  2  after merging with ending t  (13584, 17)
for df at index  3  begining_time_of_trip  (10549, 1)
for df at index  3  after merging with beginning t  (1336960, 16)
for df at index  3  ending_time_of_trip  (10549, 1)
for df at index  3  after merging with ending t  (1336960, 17)
for df at index  4  begining_time_of_trip  (125, 1)
for df at index  4  af

for df at index  35  after merging with ending t  (806756, 17)
for df at index  36  begining_time_of_trip  (168, 1)
for df at index  36  after merging with beginning t  (41458, 16)
for df at index  36  ending_time_of_trip  (168, 1)
for df at index  36  after merging with ending t  (41458, 17)
for df at index  37  begining_time_of_trip  (420, 1)
for df at index  37  after merging with beginning t  (141114, 16)
for df at index  37  ending_time_of_trip  (420, 1)
for df at index  37  after merging with ending t  (141114, 17)
for df at index  38  begining_time_of_trip  (2647, 1)
for df at index  38  after merging with beginning t  (170461, 16)
for df at index  38  ending_time_of_trip  (2647, 1)
for df at index  38  after merging with ending t  (170461, 17)
for df at index  39  begining_time_of_trip  (9806, 1)
for df at index  39  after merging with beginning t  (1097597, 16)
for df at index  39  ending_time_of_trip  (9806, 1)
for df at index  39  after merging with ending t  (1097597, 17)
f

for df at index  72  after merging with beginning t  (137845, 16)
for df at index  72  ending_time_of_trip  (945, 1)
for df at index  72  after merging with ending t  (137845, 17)
for df at index  73  begining_time_of_trip  (671, 1)
for df at index  73  after merging with beginning t  (170983, 16)
for df at index  73  ending_time_of_trip  (671, 1)
for df at index  73  after merging with ending t  (170983, 17)
for df at index  74  begining_time_of_trip  (1717, 1)
for df at index  74  after merging with beginning t  (433404, 16)
for df at index  74  ending_time_of_trip  (1717, 1)
for df at index  74  after merging with ending t  (433404, 17)
for df at index  75  begining_time_of_trip  (3745, 1)
for df at index  75  after merging with beginning t  (817274, 16)
for df at index  75  ending_time_of_trip  (3745, 1)
for df at index  75  after merging with ending t  (817274, 17)
for df at index  76  begining_time_of_trip  (3766, 1)
for df at index  76  after merging with beginning t  (675812, 1

for df at index  107  after merging with beginning t  (1171068, 16)
for df at index  107  ending_time_of_trip  (5488, 1)
for df at index  107  after merging with ending t  (1171068, 17)
for df at index  108  begining_time_of_trip  (2205, 1)
for df at index  108  after merging with beginning t  (348247, 16)
for df at index  108  ending_time_of_trip  (2205, 1)
for df at index  108  after merging with ending t  (348247, 17)
for df at index  109  begining_time_of_trip  (8, 1)
for df at index  109  after merging with beginning t  (1117, 16)
for df at index  109  ending_time_of_trip  (8, 1)
for df at index  109  after merging with ending t  (1117, 17)
for df at index  110  begining_time_of_trip  (66, 1)
for df at index  110  after merging with beginning t  (11079, 16)
for df at index  110  ending_time_of_trip  (66, 1)
for df at index  110  after merging with ending t  (11079, 17)
for df at index  111  begining_time_of_trip  (3696, 1)
for df at index  111  after merging with beginning t  (998

In [18]:
write_to_csv('/Chunks/DBUS/QP_Implementation/',chunk_dbus_list_v1)

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/239.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/151.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/70D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/1.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/123.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/61.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/161.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/33E.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/66.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/67X.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/41A.csv
Location: /home/team8/notebook

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/84.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15B.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/38D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/33D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/79A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/67.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/17A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/16D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/38B.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/46A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/29A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/185.csv
Location: /home/team8/noteb

In [19]:
chunk_dbus_list_v1[50].head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,DIFFERENCETIME_ARR,DIFFERENCETIME_DEP,LINEID,ROUTEID,DIRECTION,ACTUALTIME_ARR_DATETIME,ACTUALTIME_ARR_MIN,ACTUALTIME_ARR_MAX
0,2018-03-04,6372651,9,1444,41195,41195,41527,41527,NaN,332,332,25B,25B_274,1,2018-03-04 11:32:07,40765,43096
1,2018-03-04,6369713,9,1444,37595,37595,37649,37649,NaN,54,54,25B,25B_274,1,2018-03-04 10:27:29,37017,39106
2,2018-03-04,6376771,9,1444,42995,42995,43037,43037,NaN,42,42,25B,25B_274,1,2018-03-04 11:57:17,42346,44706
3,2018-03-04,6377267,9,1444,39395,39395,39507,39507,NaN,112,112,25B,25B_274,1,2018-03-04 10:58:27,38928,41122
4,2018-03-04,6376773,9,1444,55769,55769,55893,55900,NaN,124,131,25B,25B_274,1,2018-03-04 15:31:33,55029,57612


In [4]:
dtype={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','SUPPRESSED':'category','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','ACTUALTIME_ARR_MIN':'int64','ACTUALTIME_ARR_MAX':'int64'}
mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
arg_list=['/Chunks/DBUS/QP_Implementation/',dtype,['DAYOFSERVICE','ACTUALTIME_ARR_DATETIME'],mydateparser]

In [ ]:
chunk_dbus_list_v1=get_chunks(arg_list)

In [5]:
len(chunk_dbus_list_v1)

130

In [6]:
chunk_dbus_list_v1[40].head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,DIFFERENCETIME_ARR,DIFFERENCETIME_DEP,LINEID,ROUTEID,DIRECTION,ACTUALTIME_ARR_DATETIME,ACTUALTIME_ARR_MIN,ACTUALTIME_ARR_MAX
0,2018-12-06,8155637,1,4745,61200,61200,61308,61308,NaN,108,108,236,236_10,2,2018-12-06 17:01:48,60885,64009
1,2018-12-06,8155638,1,4745,64800,64800,64875,64875,NaN,75,75,236,236_10,2,2018-12-06 18:01:15,64472,66954
2,2018-12-06,8155636,1,4745,57600,57600,57833,57833,NaN,233,233,236,236_10,2,2018-12-06 16:03:53,57778,60326
3,2018-12-06,8155637,2,4989,61226,61226,61341,61371,NaN,115,145,236,236_10,2,2018-12-06 17:02:21,60885,64009
4,2018-12-06,8155638,2,4989,64836,64836,64905,64973,NaN,69,137,236,236_10,2,2018-12-06 18:01:45,64472,66954


In [ ]:
# (time of trip at bus stop-begining time of trip)/(ending time of trip-beginning time of trip)
for df in chunk_dbus_list_v1:
    df['TRIPS_TIME_PROPORTION']=(df['ACTUALTIME_ARR']-df['ACTUALTIME_ARR_MIN'])/(df['ACTUALTIME_ARR_MAX']-df['ACTUALTIME_ARR_MIN'])
    df['TOTAL_JOURNEY_TIME']=df['ACTUALTIME_ARR_MAX']-df['ACTUALTIME_ARR_MIN']

In [8]:
chunk_dbus_list_v1[50].head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,DIFFERENCETIME_ARR,DIFFERENCETIME_DEP,LINEID,ROUTEID,DIRECTION,ACTUALTIME_ARR_DATETIME,ACTUALTIME_ARR_MIN,ACTUALTIME_ARR_MAX,TRIPS_TIME_PROPORTION,TOTAL_JOURNEY_TIME
0,2018-03-04,6372651,9,1444,41195,41195,41527,41527,NaN,332,332,25B,25B_274,1,2018-03-04 11:32:07,40765,43096,0.326898,2331
1,2018-03-04,6369713,9,1444,37595,37595,37649,37649,NaN,54,54,25B,25B_274,1,2018-03-04 10:27:29,37017,39106,0.302537,2089
2,2018-03-04,6376771,9,1444,42995,42995,43037,43037,NaN,42,42,25B,25B_274,1,2018-03-04 11:57:17,42346,44706,0.292797,2360
3,2018-03-04,6377267,9,1444,39395,39395,39507,39507,NaN,112,112,25B,25B_274,1,2018-03-04 10:58:27,38928,41122,0.263902,2194
4,2018-03-04,6376773,9,1444,55769,55769,55893,55900,NaN,124,131,25B,25B_274,1,2018-03-04 15:31:33,55029,57612,0.334495,2583


In [ ]:
for df in chunk_dbus_list_v1:
    df.drop(['ACTUALTIME_ARR_MIN','ACTUALTIME_ARR_MAX'],axis=1,inplace=True)

In [10]:
chunk_dbus_list_v1[50].head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,DIFFERENCETIME_ARR,DIFFERENCETIME_DEP,LINEID,ROUTEID,DIRECTION,ACTUALTIME_ARR_DATETIME,TRIPS_TIME_PROPORTION,TOTAL_JOURNEY_TIME
0,2018-03-04,6372651,9,1444,41195,41195,41527,41527,NaN,332,332,25B,25B_274,1,2018-03-04 11:32:07,0.326898,2331
1,2018-03-04,6369713,9,1444,37595,37595,37649,37649,NaN,54,54,25B,25B_274,1,2018-03-04 10:27:29,0.302537,2089
2,2018-03-04,6376771,9,1444,42995,42995,43037,43037,NaN,42,42,25B,25B_274,1,2018-03-04 11:57:17,0.292797,2360
3,2018-03-04,6377267,9,1444,39395,39395,39507,39507,NaN,112,112,25B,25B_274,1,2018-03-04 10:58:27,0.263902,2194
4,2018-03-04,6376773,9,1444,55769,55769,55893,55900,NaN,124,131,25B,25B_274,1,2018-03-04 15:31:33,0.334495,2583


In [ ]:
write_to_csv('/Chunks/DBUS/QP_Implementation_backup/',chunk_dbus_list_v1)

#### Dropping supressed flagged rows

In [6]:
dtype={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','SUPPRESSED':'category','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32',"TOTAL_JOURNEY_TIME":'int16'}
mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
arg_list=['/Chunks/DBUS/QP_Implementation_backup/',dtype,['DAYOFSERVICE','ACTUALTIME_ARR_DATETIME'],mydateparser]

In [7]:
chunk_dbus_list_v2=get_chunks(arg_list)

Index at chunk_list 0  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/239.csv :  (205531, 17)
Index at chunk_list 1  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/151.csv :  (1575364, 17)
Index at chunk_list 2  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/70D.csv :  (13584, 17)
Index at chunk_list 3  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/1.csv :  (1336960, 17)
Index at chunk_list 4  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/15D.csv :  (26820, 17)
Index at chunk_list 5  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/123.csv :  (2107650, 17)
Index at chunk_list 6  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/61.csv :  (559983, 17)
Index at chunk_list 7  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/161.csv :  (39153, 17)
Index at chunk_list 8  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation

Index at chunk_list 71  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/114.csv :  (281260, 17)
Index at chunk_list 72  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/41B.csv :  (137845, 17)
Index at chunk_list 73  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/66X.csv :  (170983, 17)
Index at chunk_list 74  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/31A.csv :  (433404, 17)
Index at chunk_list 75  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/68.csv :  (817274, 17)
Index at chunk_list 76  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/184.csv :  (675812, 17)
Index at chunk_list 77  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/32.csv :  (869860, 17)
Index at chunk_list 78  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/238.csv :  (215132, 17)
Index at chunk_list 79  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Impl

In [7]:
def df_list_checker(dflist):
    print("number of df in list: ",len(chunk_dbus_list_v2))
    row_count=0
    for df in chunk_dbus_list_v2:
        row_count+=df.shape[0]
    print("number of rows of data in list: ", row_count)

In [12]:
df_list_checker(chunk_dbus_list_v2)

number of df in list:  130
number of rows of data in list:  116949113


In [14]:
chunk_dbus_list_v2[8].SUPPRESSED.unique()

[NaN, '1.0', '0.0']
Categories (2, object): ['0.0', '1.0']

In [17]:
for df in chunk_dbus_list_v2:
    df.drop(df[df.SUPPRESSED == '1.0'].index, inplace=True)
chunk_dbus_list_v2[8].SUPPRESSED.unique()

[NaN, '0.0']
Categories (2, object): ['0.0', '1.0']

In [18]:
df_list_checker(chunk_dbus_list_v2)

number of df in list:  130
number of rows of data in list:  116799395


In [19]:
for df in chunk_dbus_list_v2:
        df.drop(['SUPPRESSED'],axis=1,inplace=True)
chunk_dbus_list_v2[8].head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,DIFFERENCETIME_ARR,DIFFERENCETIME_DEP,LINEID,ROUTEID,DIRECTION,ACTUALTIME_ARR_DATETIME,TRIPS_TIME_PROPORTION,TOTAL_JOURNEY_TIME
0,2018-12-06,8155476,25,1169,48087,48087,48556,48556,469,469,15,15_17,2,2018-12-06 13:29:16,0.355314,5899
1,2018-12-06,8158340,25,1169,52287,52287,52402,52402,115,115,15,15_17,2,2018-12-06 14:33:22,0.288121,5758
2,2018-12-06,8158078,25,1169,51087,51087,51314,51325,227,238,15,15_17,2,2018-12-06 14:15:14,0.297012,6057
3,2018-12-06,8152601,25,1169,29887,29887,30029,30029,142,142,15,15_17,2,2018-12-06 08:20:29,0.310349,7440
4,2018-12-06,8158251,25,1169,31687,31687,31644,31644,-43,-43,15,15_17,2,2018-12-06 08:47:24,0.306390,7027


In [22]:
write_to_csv('/Chunks/DBUS/QP_Implementation/',chunk_dbus_list_v2)

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/239.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/151.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/70D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/1.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/123.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/61.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/161.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/33E.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/66.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/67X.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/41A.csv
Location: /home/team8/notebook

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/84.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15B.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/38D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/33D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/79A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/67.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/17A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/16D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/38B.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/46A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/29A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/185.csv
Location: /home/team8/noteb

In [3]:
dtype={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32',"TOTAL_JOURNEY_TIME":'int16'}
mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
arg_list=['/Chunks/DBUS/QP_Implementation/',dtype,['DAYOFSERVICE','ACTUALTIME_ARR_DATETIME'],mydateparser]

In [4]:
chunk_dbus_list_v2=get_chunks(arg_list)

Index at chunk_list 0  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/239.csv :  (205488, 16)
Index at chunk_list 1  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/151.csv :  (1574006, 16)
Index at chunk_list 2  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/70D.csv :  (13584, 16)
Index at chunk_list 3  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/1.csv :  (1335756, 16)
Index at chunk_list 4  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15D.csv :  (26752, 16)
Index at chunk_list 5  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/123.csv :  (2104705, 16)
Index at chunk_list 6  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/61.csv :  (559566, 16)
Index at chunk_list 7  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/161.csv :  (39055, 16)
Index at chunk_list 8  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15.csv :  (3899203, 16)
Index at chunk_list 9  is  /hom

Index at chunk_list 76  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/184.csv :  (675618, 16)
Index at chunk_list 77  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/32.csv :  (869517, 16)
Index at chunk_list 78  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/238.csv :  (214866, 16)
Index at chunk_list 79  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/40D.csv :  (1162497, 16)
Index at chunk_list 80  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/84X.csv :  (174802, 16)
Index at chunk_list 81  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/120.csv :  (813550, 16)
Index at chunk_list 82  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/47.csv :  (643496, 16)
Index at chunk_list 83  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/44.csv :  (919855, 16)
Index at chunk_list 84  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/68X.csv :  (9845, 16)
Index at chunk_list 85 

In [132]:
for df in chunk_dbus_list_v2:
    # Check for duplicate rows. 
    # Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
    print('='*66)
    print("LINEID is ",df['LINEID'].unique()[0])
    print('Number of duplicate rows (including first) in the table is:', df[df.duplicated(keep=False)].shape[0])

LINEID is  239
Number of duplicate rows (including first) in the table is: 0
LINEID is  151
Number of duplicate rows (including first) in the table is: 0
LINEID is  70D
Number of duplicate rows (including first) in the table is: 0
LINEID is  1
Number of duplicate rows (including first) in the table is: 0
LINEID is  15D
Number of duplicate rows (including first) in the table is: 0
LINEID is  123
Number of duplicate rows (including first) in the table is: 0
LINEID is  61
Number of duplicate rows (including first) in the table is: 0
LINEID is  161
Number of duplicate rows (including first) in the table is: 0
LINEID is  15
Number of duplicate rows (including first) in the table is: 0
LINEID is  33E
Number of duplicate rows (including first) in the table is: 0
LINEID is  66
Number of duplicate rows (including first) in the table is: 0
LINEID is  67X
Number of duplicate rows (including first) in the table is: 0
LINEID is  41A
Number of duplicate rows (including first) in the table is: 0
LINE

Number of duplicate rows (including first) in the table is: 0
LINEID is  27
Number of duplicate rows (including first) in the table is: 0
LINEID is  31D
Number of duplicate rows (including first) in the table is: 0
LINEID is  142
Number of duplicate rows (including first) in the table is: 0
LINEID is  46E
Number of duplicate rows (including first) in the table is: 0
LINEID is  33A
Number of duplicate rows (including first) in the table is: 0
LINEID is  7D
Number of duplicate rows (including first) in the table is: 0
LINEID is  16
Number of duplicate rows (including first) in the table is: 0
LINEID is  56A
Number of duplicate rows (including first) in the table is: 0
LINEID is  76A
Number of duplicate rows (including first) in the table is: 0
LINEID is  9
Number of duplicate rows (including first) in the table is: 0
LINEID is  75
Number of duplicate rows (including first) in the table is: 0
LINEID is  42
Number of duplicate rows (including first) in the table is: 0
LINEID is  33B
Number

Number of duplicate rows (including first) in the table is: 0
LINEID is  67
Number of duplicate rows (including first) in the table is: 0
LINEID is  15A
Number of duplicate rows (including first) in the table is: 0
LINEID is  17A
Number of duplicate rows (including first) in the table is: 0
LINEID is  16D
Number of duplicate rows (including first) in the table is: 0
LINEID is  38B
Number of duplicate rows (including first) in the table is: 0
LINEID is  46A
Number of duplicate rows (including first) in the table is: 0
LINEID is  29A
Number of duplicate rows (including first) in the table is: 0
LINEID is  185
Number of duplicate rows (including first) in the table is: 0
LINEID is  69X
Number of duplicate rows (including first) in the table is: 0
LINEID is  44B
Number of duplicate rows (including first) in the table is: 0
LINEID is  39
Number of duplicate rows (including first) in the table is: 0
LINEID is  83A
Number of duplicate rows (including first) in the table is: 0
LINEID is  40E
N

In [12]:
for df in chunk_dbus_list_v2:
    print('='*66)
    print("LINEID is ",df['LINEID'].unique()[0])
    print(df.isnull().sum())

LINEID is  239
DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      2
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  151
DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      5
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  70D
D

LINEID is  102
DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      0
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  40B
DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      0
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  77X
D

DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      0
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  13
DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      8
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  38
DAYOFSERVICE      

DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      2
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  31D
DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      0
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  142
DAYOFSERVICE    

DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      1
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  32
DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      0
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  238
DAYOFSERVICE     

LINEID is  41
DAYOFSERVICE                0
TRIPID                      0
PROGRNUMBER                 0
STOPPOINTID                 0
PLANNEDTIME_ARR             0
PLANNEDTIME_DEP             0
ACTUALTIME_ARR              0
ACTUALTIME_DEP              0
DIFFERENCETIME_ARR          0
DIFFERENCETIME_DEP          0
LINEID                      0
ROUTEID                     0
DIRECTION                   0
ACTUALTIME_ARR_DATETIME     0
TRIPS_TIME_PROPORTION      22
TOTAL_JOURNEY_TIME          0
dtype: int64
LINEID is  76
DAYOFSERVICE                0
TRIPID                      0
PROGRNUMBER                 0
STOPPOINTID                 0
PLANNEDTIME_ARR             0
PLANNEDTIME_DEP             0
ACTUALTIME_ARR              0
ACTUALTIME_DEP              0
DIFFERENCETIME_ARR          0
DIFFERENCETIME_DEP          0
LINEID                      0
ROUTEID                     0
DIRECTION                   0
ACTUALTIME_ARR_DATETIME     0
TRIPS_TIME_PROPORTION      14
TOTAL_JOURNEY_TIME          0

LINEID is  33D
DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      0
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  79A
DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
DIFFERENCETIME_ARR         0
DIFFERENCETIME_DEP         0
LINEID                     0
ROUTEID                    0
DIRECTION                  0
ACTUALTIME_ARR_DATETIME    0
TRIPS_TIME_PROPORTION      3
TOTAL_JOURNEY_TIME         0
dtype: int64
LINEID is  67
DA

At above check we noticed for some columns trips time proportion is missing. Remember time proportion include division to difference between beginning and ending of trips times. Which means divider was 0 for those rows, which points out to logical integrity faulty rows, so we will drop them 

In [5]:
is_null_counter=0
for df in chunk_dbus_list_v2:
    is_null_counter+=df.TRIPS_TIME_PROPORTION.isnull().sum()
is_null_counter

442

In [6]:
for df in chunk_dbus_list_v2:
    df.dropna(subset=['TRIPS_TIME_PROPORTION'],inplace=True)

number of rows of data in list before drop: 116799395  after drop 116798953 ;  116799395-116798953=442 rows dropped

In [8]:
df_list_checker(chunk_dbus_list_v2)

number of df in list:  130
number of rows of data in list:  116798953


#### Extract "time of day", "day of week", "months" from  ACTUALTIME_ARR_DATETIME	

In [9]:
for df in chunk_dbus_list_v2:
    df['TIME_OF_DAY']=df['ACTUALTIME_ARR_DATETIME'].dt.strftime('%H')
    df['DAY_OF_WEEK']=df['ACTUALTIME_ARR_DATETIME'].dt.strftime('%d')
    df['MONTH']=df['ACTUALTIME_ARR_DATETIME'].dt.strftime('%m')

In [10]:
chunk_dbus_list_v2[8].head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,DIFFERENCETIME_ARR,DIFFERENCETIME_DEP,LINEID,ROUTEID,DIRECTION,ACTUALTIME_ARR_DATETIME,TRIPS_TIME_PROPORTION,TOTAL_JOURNEY_TIME,TIME_OF_DAY,DAY_OF_WEEK,MONTH
0,2018-12-06,8155476,25,1169,48087,48087,48556,48556,469,469,15,15_17,2,2018-12-06 13:29:16,0.355314,5899,13,06,12
1,2018-12-06,8158340,25,1169,52287,52287,52402,52402,115,115,15,15_17,2,2018-12-06 14:33:22,0.288121,5758,14,06,12
2,2018-12-06,8158078,25,1169,51087,51087,51314,51325,227,238,15,15_17,2,2018-12-06 14:15:14,0.297012,6057,14,06,12
3,2018-12-06,8152601,25,1169,29887,29887,30029,30029,142,142,15,15_17,2,2018-12-06 08:20:29,0.310349,7440,08,06,12
4,2018-12-06,8158251,25,1169,31687,31687,31644,31644,-43,-43,15,15_17,2,2018-12-06 08:47:24,0.306390,7027,08,06,12


In [13]:
write_to_csv('/Chunks/DBUS/QP_Implementation_backup/',chunk_dbus_list_v2)

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/239.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/151.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/70D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/1.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/15D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/123.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/61.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/161.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/15.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/33E.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/66.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/67X.csv
Location: /home/team8

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/51D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/25D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/27A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/33.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/40.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/31B.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/17.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/111.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/41D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/42D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/84.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/15B.csv
Location: /home/team

In [3]:
dtype={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category','MONTH':'category'}
mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
arg_list=['/Chunks/DBUS/QP_Implementation_backup/',dtype,['DAYOFSERVICE','ACTUALTIME_ARR_DATETIME'],mydateparser]

In [5]:
chunk_dbus_list_v2=get_chunks(arg_list)

Index at chunk_list 0  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/239.csv :  (205486, 19)
Index at chunk_list 1  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/151.csv :  (1574001, 19)
Index at chunk_list 2  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/70D.csv :  (13584, 19)
Index at chunk_list 3  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/1.csv :  (1335752, 19)
Index at chunk_list 4  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/15D.csv :  (26752, 19)
Index at chunk_list 5  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/123.csv :  (2104694, 19)
Index at chunk_list 6  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/61.csv :  (559566, 19)
Index at chunk_list 7  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/161.csv :  (39055, 19)
Index at chunk_list 8  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation

Index at chunk_list 71  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/114.csv :  (281184, 19)
Index at chunk_list 72  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/41B.csv :  (137593, 19)
Index at chunk_list 73  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/66X.csv :  (170945, 19)
Index at chunk_list 74  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/31A.csv :  (433162, 19)
Index at chunk_list 75  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/68.csv :  (817008, 19)
Index at chunk_list 76  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/184.csv :  (675617, 19)
Index at chunk_list 77  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/32.csv :  (869517, 19)
Index at chunk_list 78  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation_backup/238.csv :  (214865, 19)
Index at chunk_list 79  is  /home/team8/notebook/Pelin/Chunks/DBUS/QP_Impl

In [8]:
df_list_checker(chunk_dbus_list_v2)

number of df in list:  130
number of rows of data in list:  116798953


In [9]:
for df in chunk_dbus_list_v2:
    df.drop(['DAYOFSERVICE','MONTH'],axis=1,inplace=True)

In [12]:
write_to_csv('/Chunks/DBUS/QP_Implementation/',chunk_dbus_list_v2)

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/239.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/151.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/70D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/1.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/123.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/61.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/161.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/33E.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/66.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/67X.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/41A.csv
Location: /home/team8/notebook

Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/84.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15B.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/38D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/33D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/79A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/67.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/15A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/17A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/16D.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/38B.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/46A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/29A.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/QP_Implementation/185.csv
Location: /home/team8/noteb

In [ ]:
dtype={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category'}
mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
arg_list=['/Chunks/DBUS/QP_Implementation/',dtype,['ACTUALTIME_ARR_DATETIME'],mydateparser]

In [ ]:
chunk_dbus_list_v2=get_chunks(arg_list)

#### Check to see if there is increasing trend between ACTUALTIME_ARR increase as PROGRNUMBER.

#### Check interactions of LINEID ROUTEID DIRECTION STOPPOINTID PROGRNUMBER  

Interaction between PROGRNUMBER and LINEID ROUTEID DIRECTION STOPPOINTID

In [16]:
PROGRNUMBER_GB_STOPPOINTID_ROUTEID=[]
for df in chunk_dbus_list_v2:
    print('='*66)
    print("LINEID is ",df['LINEID'].unique()[0])
    print("There is ", df['ROUTEID'].nunique(), " ROUTEID for this LINEID.")
    print("There is ", df['STOPPOINTID'].nunique(), " STOPPOINTID for this LINEID.")    
    print("There is ", df['TRIPID'].nunique(), " TRIPID for this LINEID.")  
    print("'STOPPOINTID's have maksimum ", (df.groupby(['STOPPOINTID']))['DIRECTION'].nunique().max(), " unique 'DIRECTION' value for this LINEID." )
    print("'STOPPOINTID's have maksimum ",(df.groupby(['STOPPOINTID']))['ROUTEID'].nunique().max() ," unique 'ROUTEID' value for this LINEID." )
    print("'STOPPOINTID','DIRECTION' combinations have maksimum ",(df.groupby(['STOPPOINTID','DIRECTION']))['PROGRNUMBER'].nunique().max()," unique 'PROGRNUMBER' value for this LINEID." )
    x=(df.groupby(['STOPPOINTID','ROUTEID']))['PROGRNUMBER'].nunique().max()
    print("'STOPPOINTID','ROUTEID' combinations have maksimum ",x ," unique 'PROGRNUMBER' value for this LINEID." )
    PROGRNUMBER_GB_STOPPOINTID_ROUTEID.append(x)
    print("'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum ",(df.groupby(['STOPPOINTID','ROUTEID','DIRECTION']))['PROGRNUMBER'].nunique().max() ," unique 'PROGRNUMBER' value for this LINEID." )
print('='*66)
print("Maksimum unique number of PROGRNUMBER grouped by STOPPOINTID and ROUTEID is ", max(PROGRNUMBER_GB_STOPPOINTID_ROUTEID))

LINEID is  239
There is  2  ROUTEID for this LINEID.
There is  60  STOPPOINTID for this LINEID.
There is  1873  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  151
There is  5  ROUTEID for this LINEID.
There is  101  STOPPOINTID for this LINEID.
There is  10303  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  1  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  3  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  2  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROU

LINEID is  11
There is  4  ROUTEID for this LINEID.
There is  125  STOPPOINTID for this LINEID.
There is  8413  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  3  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  25
There is  2  ROUTEID for this LINEID.
There is  82  STOPPOINTID for this LINEID.
There is  3308  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEI

'STOPPOINTID','ROUTEID' combinations have maksimum  2  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  2  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  66A
There is  4  ROUTEID for this LINEID.
There is  93  STOPPOINTID for this LINEID.
There is  3216  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  4  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  14C
There is  3  ROUTEID for this LINEID.
There is  77  STOPPOINTID for this LINEID.
There is  310  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  1  unique 'DIRECTION' value fo

'STOPPOINTID's have maksimum  1  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  2  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  236
There is  2  ROUTEID for this LINEID.
There is  42  STOPPOINTID for this LINEID.
There is  247  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maks

'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  32X
There is  3  ROUTEID for this LINEID.
There is  102  STOPPOINTID for this LINEID.
There is  168  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  3  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  145
There is  15  ROUTEID for this LINEID.
There is  146  STOPPOINTID for this LINEID.
There is  17676  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'DIRECTION' valu

...

'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  32
There is  2  ROUTEID for this LINEID.
There is  116  STOPPOINTID for this LINEID.
There is  4429  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  1  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  1  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  238
There is  3  ROUTEID for this LINEID.
There is  47  STOPPOINTID for this LINEID.
There is  3130  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  1  unique 'DIRECTION' value f

'STOPPOINTID's have maksimum  6  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  3  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  38A
There is  4  ROUTEID for this LINEID.
There is  110  STOPPOINTID for this LINEID.
There is  5340  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  4  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  2  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  68A
There

'STOPPOINTID's have maksimum  3  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  2  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  33
There is  7  ROUTEID for this LINEID.
There is  216  STOPPOINTID for this LINEID.
There is  4585  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  6  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  5  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  2  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  2  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  40
There i

'STOPPOINTID's have maksimum  1  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  1  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
LINEID is  15A
There is  3  ROUTEID for this LINEID.
There is  83  STOPPOINTID for this LINEID.
There is  10072  TRIPID for this LINEID.
'STOPPOINTID's have maksimum  1  unique 'DIRECTION' value for this LINEID.
'STOPPOINTID's have maksimum  2  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  2  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have ma

'STOPPOINTID's have maksimum  3  unique 'ROUTEID' value for this LINEID.
'STOPPOINTID','DIRECTION' combinations have maksimum  2  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
'STOPPOINTID','ROUTEID','DIRECTION' combinations have maksimum  1  unique 'PROGRNUMBER' value for this LINEID.
Maksimum unique number of PROGRNUMBER grouped by STOPPOINTID and ROUTEID is  3


From above information we can see that;
* ROUTEID contains information about direction therefore use of this parameter eleminates need for direction category.
* 'STOPPOINTID','ROUTEID' can have more then 1 unique program number. 
    

Interaction between LINEID  ROUTEID and STOPPOINTID

In [10]:
for df in chunk_dbus_list_v2:
    x= (df.groupby(['STOPPOINTID']))['ROUTEID'].nunique().max()
    if x >1:
        print('='*66)
        print("LINEID is ",df['LINEID'].unique()[0])
        print("STOPPOINTID group by ROUTEID value counts  this line:")
        a=(df.groupby(['STOPPOINTID']))['ROUTEID'].value_counts(dropna=True)
        print(a.loc[a>0])

LINEID is  239
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
1844         239_28    3046
1882         239_26    3207
1883         239_26    3186
1884         239_26    3054
1887         239_26    3122
                       ... 
7026         239_26    3212
7155         239_26    3049
7168         239_28    3419
7379         239_26    3218
7563         239_26    3223
Name: ROUTEID, Length: 63, dtype: int64
LINEID is  151
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
1406         151_15    12013
             151_16     3294
1407         151_15    12013
             151_16     3296
1409         151_15    12017
                       ...  
7622         151_17    13648
             151_18     2268
             151_19      292
7623         151_17    13334
             151_18     2244
Name: ROUTEID, Length: 243, dtype: int64
LINEID is  70D
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
115          70D_70    17

LINEID is  27B
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
1176         27B_27      479
             27B_38      222
1177         27B_27      465
             27B_38      231
1178         27B_27      480
                       ...  
992          27B_34    11503
             27B_35     3208
993          27B_34    11562
             27B_35     3217
             27B_36      886
Name: ROUTEID, Length: 428, dtype: int64
LINEID is  102
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
1039         102_8     9360
1040         102_8     9362
1041         102_9     9021
1050         102_9     9052
1073         102_9     9063
                       ... 
951          102_10    1084
952          102_9     8874
             102_10    1038
953          102_9     8875
             102_10    1038
Name: ROUTEID, Length: 138, dtype: int64
LINEID is  40B
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
11           40B_63    1

LINEID is  54A
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
100          54A_12    9116
1096         54A_11    9083
1097         54A_11    9082
1099         54A_11    9087
1100         54A_12    9117
                       ... 
5161         54A_12    8896
5162         54A_12    9075
6132         54A_11    9083
6170         54A_11    9001
7214         54A_11    8978
Name: ROUTEID, Length: 89, dtype: int64
LINEID is  27X
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
1196         27X_43    446
1197         27X_43    446
1198         27X_43    447
1199         27X_43    447
1200         27X_43    447
                      ... 
664          27X_42    471
7490         27X_43    447
763          27X_42    468
7659         27X_43    444
768          27X_43    441
Name: ROUTEID, Length: 93, dtype: int64
LINEID is  33X
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
2498         33X_46    969
             33X_48  

LINEID is  14
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID       
1016         14_15    17881
1017         14_15    17880
1018         14_15    17886
1019         14_15    17887
1020         14_15    17888
                      ...  
7582         14_16    17115
7589         14_16    17132
7659         14_16    17169
             14_12      817
             14_13       46
Name: ROUTEID, Length: 250, dtype: int64
LINEID is  32X
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
1729         32X_77    414
             32X_74     67
1731         32X_77    414
             32X_74     67
1738         32X_77    414
                      ... 
947          32X_74     67
948          32X_77    413
             32X_74     67
949          32X_77    413
             32X_74     67
Name: ROUTEID, Length: 145, dtype: int64
LINEID is  145
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID         
1443         145_105    25846
             145_

STOPPOINTID       
1184         42_44    12564
             42_42    11998
1196         42_42    11535
1197         42_42    11552
1198         42_42    11576
                      ...  
7541         42_44        5
7542         42_44        3
7616         42_44    12272
7659         42_42    11514
905          42_44    12246
Name: ROUTEID, Length: 121, dtype: int64
LINEID is  33B
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
3679         33B_56    6567
3689         33B_58    6366
             33B_60       9
3690         33B_58    6355
             33B_60       9
                       ... 
7204         33B_56    5675
             33B_59      68
             33B_60       9
7594         33B_58    6366
             33B_60       9
Name: ROUTEID, Length: 101, dtype: int64
LINEID is  114
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID       
2833         114_6    4921
3082         114_6    4913
3085         114_5    4280
             114_6    427

STOPPOINTID       
2007         116_1    192
2008         116_1    192
2009         116_1    192
2010         116_1    192
2011         116_1    193
                     ... 
848          116_1    121
906          116_3    235
907          116_3    235
908          116_3    235
909          116_3    235
Name: ROUTEID, Length: 115, dtype: int64
LINEID is  41C
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
1171         41C_78    12117
             41C_83      891
1172         41C_78    12176
             41C_83      894
1173         41C_78    12374
                       ...  
7603         41C_84      982
7655         41C_79       24
             41C_84        4
85           41C_78    12446
             41C_83      931
Name: ROUTEID, Length: 283, dtype: int64
LINEID is  16C
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
119          16C_29    568
             16C_26    450
             16C_27    227
1325         16C_28    661
1326    

STOPPOINTID       
1171         33_44    3426
             33_45    2571
             33_69      79
1172         33_44    3665
             33_45    2725
                      ... 
7690         33_44     295
             33_45     126
85           33_44    4312
             33_45    3268
             33_69     115
Name: ROUTEID, Length: 692, dtype: int64
LINEID is  40
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID       
11           40_31    26392
             40_33      280
1280         40_31    25841
             40_33      245
1358         40_31    26883
                      ...  
840          40_28      373
841          40_27    26178
             40_28      373
987          40_31    25962
             40_33      244
Name: ROUTEID, Length: 395, dtype: int64
LINEID is  31B
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
289          31B_44    713
4384         31B_44    712
496          31B_44    711
5062         31B_44    654
          

LINEID is  39
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID       
1443         39_20    11494
             39_22      349
1444         39_20    11564
             39_22      349
1445         39_20    11615
                      ...  
785          39_22      346
786          39_20    10916
             39_22      347
793          39_20    10823
             39_22      341
Name: ROUTEID, Length: 302, dtype: int64
LINEID is  83A
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID        
1016         83A_17    3770
             83A_20     797
1017         83A_17    3771
             83A_20     798
1018         83A_17    3773
                       ... 
7500         83A_17     185
7501         83A_17     101
7502         83A_17      22
7503         83A_17      32
7504         83A_17       1
Name: ROUTEID, Length: 264, dtype: int64
LINEID is  49
STOPPOINTID group by ROUTEID value counts  this line:
STOPPOINTID       
1121         49_27    10862
1122       

In [14]:
for df in chunk_dbus_list_v2:
    x= (df.groupby(['STOPPOINTID','ROUTEID']))['PROGRNUMBER'].nunique().max()
    if x >1:
        print('='*66)
        print("LINEID is ",df['LINEID'].unique()[0])
        print("PROGRNUMBER value counts for'STOPPOINTID''DIRECTION'combination for this line:")
        print((df.groupby(['STOPPOINTID','ROUTEID']))['PROGRNUMBER'].value_counts(dropna=False))

LINEID is  70D
PROGRNUMBER value counts for'STOPPOINTID''DIRECTION'combination for this line:
STOPPOINTID  ROUTEID  PROGRNUMBER
115          70D_70   4              178
1510         70D_71   33             103
                      32              78
1545         70D_71   10             180
156          70D_71   34             103
                                    ... 
7571         70D_71   40              76
90           70D_71   39             103
                      38              78
91           70D_71   40             103
                      39              78
Name: PROGRNUMBER, Length: 86, dtype: int64
LINEID is  104
PROGRNUMBER value counts for'STOPPOINTID''DIRECTION'combination for this line:
STOPPOINTID  ROUTEID  PROGRNUMBER
114          104_15   33             3541
                      31              240
1196         104_15   17             3543
                      15              243
1197         104_15   18             3547
                                     ..

LINEID is  142
PROGRNUMBER value counts for'STOPPOINTID''DIRECTION'combination for this line:
STOPPOINTID  ROUTEID  PROGRNUMBER
1013         142_13   34              687
                      35               26
1015         142_13   35              687
                      36               26
1016         142_10   34              156
                                     ... 
895          142_13   47              667
                      48               26
             142_8    12             1390
905          142_10   15              163
             142_13   15              831
Name: PROGRNUMBER, Length: 231, dtype: int64
LINEID is  16
PROGRNUMBER value counts for'STOPPOINTID''DIRECTION'combination for this line:
STOPPOINTID  ROUTEID  PROGRNUMBER
119          16_20    25             20370
             16_21    12              1802
             16_22    9                240
             16_23    11               901
12           16_24    48             22859
                       

LINEID is  33
PROGRNUMBER value counts for'STOPPOINTID''DIRECTION'combination for this line:
STOPPOINTID  ROUTEID  PROGRNUMBER
1171         33_44    2              3426
             33_45    2              2571
             33_69    2                79
1172         33_44    3              3665
             33_45    3              2725
                                     ... 
7690         33_44    67              295
             33_45    67              126
85           33_44    13             4312
             33_45    13             3268
             33_69    13              115
Name: PROGRNUMBER, Length: 1020, dtype: int64
LINEID is  17
PROGRNUMBER value counts for'STOPPOINTID''DIRECTION'combination for this line:
STOPPOINTID  ROUTEID  PROGRNUMBER
1035         17_15    30             4797
                      28             3080
             17_16    27              459
             17_17    18              361
                      16              215
                            

#### Dividing data into ROUTEID chunks

In [2]:
#https://stackoverflow.com/questions/70911236/how-can-i-concat-the-huge-pandas-dataframes-into-one
def yield_dfs():
    cwd=os.getcwd()
    dtypes={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category'}
    mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
    x=['/Chunks/DBUS/QP_Implementation',dtypes,['ACTUALTIME_ARR_DATETIME'],mydateparser]
    path = cwd+x[0]
    file_list = glob.glob(os.path.join(path, "*.csv"))
    """generator function to yield dataframes."""
    for filename in file_list:
        df = pd.read_csv(filename, low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3],index_col=False)
        print(df.shape)
        yield df

df_dbus = pd.concat(yield_dfs(), axis=0,ignore_index=True)

(205486, 19)
(1574001, 19)
(13584, 19)
(1335752, 19)
(26752, 19)
(2104694, 19)
(559566, 19)
(39055, 19)
(3899194, 19)
(1452, 19)
(1259753, 19)
(219785, 19)
(24588, 19)
(1651625, 19)
(420686, 19)
(119987, 19)
(1307193, 19)
(265585, 19)
(48390, 19)
(1447481, 19)
(997218, 19)
(142823, 19)
(15523, 19)
(2267188, 19)
(195969, 19)
(732713, 19)
(468109, 19)
(38374, 19)
(3751761, 19)
(910121, 19)
(484586, 19)
(802720, 19)
(666030, 19)
(11179, 19)
(49059, 19)
(806194, 19)
(41458, 19)
(141083, 19)
(170389, 19)
(1096479, 19)
(31347, 19)
(3358961, 19)
(1159871, 19)
(406087, 19)
(1917279, 19)
(1439949, 19)
(462938, 19)
(1132879, 19)
(2080114, 19)
(86067, 19)
(1094941, 19)
(2673250, 19)
(44087, 19)
(4113804, 19)
(142930, 19)
(703917, 19)
(1862215, 19)
(1263849, 19)
(4156730, 19)
(12126, 19)
(146901, 19)
(17918, 19)
(228656, 19)
(41540, 19)
(3370954, 19)
(532428, 19)
(55435, 19)
(2766912, 19)
(1465109, 19)
(1383307, 19)
(313819, 19)
(281184, 19)
(137593, 19)
(170945, 19)
(433162, 19)
(817008, 19)
(675

In [4]:
df_dbus.shape

(116798953, 19)

In [3]:
df_ROUTEID = pd.DataFrame(df_dbus['ROUTEID'].unique(), columns=['ROUTEID'])
df_ROUTEID.to_csv("ROUTEID.csv", index=False)

In [6]:
ROUTEID_file = "ROUTEID.csv"
df_ROUTEID = pd.read_csv(ROUTEID_file, low_memory=False)

In [7]:
ROUTEID_array=df_ROUTEID['ROUTEID'].unique()
ROUTEID_array.shape

(588,)

In [8]:
ROUTEID_list=ROUTEID_array.tolist()
len(ROUTEID_list)

588

In [16]:
#creating directory structure
def create_dir_w_list(adress, alist):
    counter=0
    for df in alist:
        cwd=os.getcwd()
        directory= "ROUTEID_" + alist[counter]
        counter+=1
        parent_dir= cwd+adress
        path = os.path.join(parent_dir, directory)
        os.mkdir(path)

In [17]:
create_dir_w_list('/Chunks/DBUS/ROUTEID_CHUNKS/',ROUTEID_list)

Selecting data in each chunk, writing to csv and loading into folders marked by ROUTEID

In [3]:
cwd=os.getcwd()

In [19]:
ROUTEID_chunk_list=[]
for value in ROUTEID_list:
    counter=0
    for df in chunk_dbus_list_v2:
        word=df['LINEID'].unique()[0]
        string=value
        if word in string:
            ROUTEID_chunk=df.loc[df['ROUTEID'] == value]
            adress='/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_'+value+'/'
            path = cwd+adress
            ROUTEID_chunk.to_csv(path+value+str(counter)+'.csv',index=False)
            counter+=1
            ROUTEID_chunk_list.append(ROUTEID_chunk)
            print(path+value+str(counter))

/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_239_26/239_261
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_239_26/239_262
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_239_26/239_263
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_239_26/239_264
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_239_28/239_281
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_239_28/239_282
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_239_28/239_283
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_151_18/151_181
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_151_18/151_182
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_151_18/151_183
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_151_18/151_184
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_151_19/151_191
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_151_19/151_192

/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_33E_74/33E_741
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_33E_74/33E_742
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_33E_74/33E_743
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_33E_74/33E_744
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66_17/66_171
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66_17/66_172
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66_17/66_173
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66_17/66_174
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66_18/66_181
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66_18/66_182
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66_18/66_183
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66_14/66_141
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66_14/66_142
/home/team8/noteb

/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_7_72/7_721
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_7_70/7_701
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_7_70/7_702
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_7_71/7_711
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_7_71/7_712
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_7_66/7_661
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_7_66/7_662
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_7_67/7_671
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_7_67/7_672
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_104_15/104_151
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_104_15/104_152
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_104_15/104_153
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_104_15/104_154
/home/team8/notebook/Pelin/Chunks/D

/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_77A_28/77A_281
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_77A_28/77A_282
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_77A_28/77A_283
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_77A_30/77A_301
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_77A_30/77A_302
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_77A_30/77A_303
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_77A_31/77A_311
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_77A_31/77A_312
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_77A_31/77A_313
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_77A_31/77A_314
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_77A_31/77A_315
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_270_42/270_421
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_270_42/270_422

/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66B_59/66B_592
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66B_59/66B_593
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66B_59/66B_594
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66B_55/66B_551
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66B_55/66B_552
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66B_57/66B_571
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66B_57/66B_572
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_66B_57/66B_573
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_43_84/43_841
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_43_84/43_842
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_43_84/43_843
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_43_85/43_851
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_43_85/43_852
/home/tea

...

/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_69_47/69_473
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_69_47/69_474
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_69_47/69_475
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41_7/41_71
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41_7/41_72
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41_7/41_73
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41_7/41_74
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41_10/41_101
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41_10/41_102
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41_10/41_103
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41_3/41_31
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41_3/41_32
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41_3/41_33
/home/team8/notebook/Pelin/Chunks/DBUS/

/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_27A_5/27A_53
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_27A_5/27A_54
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_27A_6/27A_61
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_27A_6/27A_62
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_27A_6/27A_63
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_27A_6/27A_64
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_27A_7/27A_71
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_27A_7/27A_72
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_27A_7/27A_73
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_27A_7/27A_74
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_33_72/33_721
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_33_72/33_722
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_33_44/33_441
/home/team8/notebook/Peli

/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_111_8/111_83
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_111_10/111_101
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_111_10/111_102
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_111_10/111_103
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41D_27/41D_271
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41D_27/41D_272
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41D_27/41D_273
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41D_27/41D_274
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41D_27/41D_275
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41D_27/41D_276
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41D_26/41D_261
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41D_26/41D_262
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_41D_26/41D_263
/

/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_16/17A_163
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_16/17A_164
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_16/17A_165
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_16/17A_166
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_18/17A_181
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_18/17A_182
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_18/17A_183
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_18/17A_184
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_18/17A_185
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_18/17A_186
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_13/17A_131
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_13/17A_132
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_17A_13/17A_133

/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_56/185_563
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_50/185_501
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_50/185_502
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_50/185_503
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_62/185_621
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_62/185_622
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_62/185_623
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_54/185_541
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_54/185_542
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_54/185_543
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_54/185_544
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_51/185_511
/home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_185_51/185_512

Merging csvs in each ROUTEID folder to create dataframe for same ROUTEID and saving dataframe to a csv file named with ROUTEID.

**runs & remarks for data integrity checks**: 

    run1: 63_31.csv was the last file created before kernel died, will be cleaned and re-created after all chunks created. (DONE)

In [10]:
ROUTEID_df_list=[]
for value in ROUTEID_list:
    cwd=os.getcwd()
    adress2='/Chunks/DBUS/ROUTEID_CHUNKS_CVS/'
    path2 = cwd+adress2
    file_name=value+'.csv'
    my_file_bool=os.path.exists(path2+file_name)
    if my_file_bool==False:
        #if value.csv is not in path2 do below code.
        adress='/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_'+value+'/'
        dtypes={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category'}
        mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
        arg_list=[adress,dtypes,['ACTUALTIME_ARR_DATETIME'],mydateparser]
        df_list_ROUTEID_CHUNKS_folder_content=get_chunks(arg_list)
        df_ROUTEID=pd.concat(df_list_ROUTEID_CHUNKS_folder_content, axis=0)
        ROUTEID_df_list.append(df_ROUTEID)
        df_ROUTEID.to_csv(path2+file_name,index=False)
        print('Location:', path2+file_name)

Index at chunk_list 0  is  /home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_63_31/63_310.csv :  (0, 17)
Index at chunk_list 1  is  /home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_63_31/63_312.csv :  (8154, 17)
Index at chunk_list 2  is  /home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS/ROUTEID_63_31/63_311.csv :  (0, 17)
Location: /home/team8/notebook/Pelin/Chunks/DBUS/ROUTEID_CHUNKS_CVS/63_31.csv


In [ ]:
for df in chunk_dbus_list:
    df['ACTUALTIME_ARR_DATETIME'] = pd.to_datetime(df['DAYOFSERVICE']) + pd.to_timedelta(df['ACTUALTIME_ARR'], unit='s')